Installing needed libraries


In [12]:
pip install google-cloud-bigquery

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pandas-gbq


Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install db-dtypes


Note: you may need to restart the kernel to use updated packages.


In [15]:
from google.cloud import bigquery

# Path to your JSON key file
key_path = "/Users/itsme/Desktop/WEBEET/Week2/bq_connector_key.json" 
client = bigquery.Client.from_service_account_json(key_path)



Reading table that is already there

In [16]:
import pandas as pd 

query = """
    SELECT *
    FROM `cohesive-idiom-441010-a8.test.my_new_table`
    LIMIT 5
"""

# Read data using pandas
df = pd.read_gbq(
    query,
    project_id="cohesive-idiom-441010-a8",
    credentials=client._credentials  # reuse credentials from your bigquery.Client
)

print(df)

   id     name  score
0   1    Alice     85
1   2      Bob     90
2   3  Charlie     95
3   1    Alice     85
4   2      Bob     90


How to create new table in BigQuery

In [17]:
#Load the cleaned U-Bahn dataset and explicitly set 'postcode' as string (better for preserving leading zeros and formatting)
df = pd.read_csv("ubahn.csv", dtype={"postcode": str})

# Rename columns: 'stadtteil' to 'neighborhood', 'neighborhood' to 'district' for consistency and clarity
df.rename(columns={"stadtteil": "neighborhood", "neighborhood": "district"}, inplace=True)

In [18]:
# Map districts to their IDs
district_id_map = {
    "Mitte": "01",
    "Friedrichshain-Kreuzberg": "02",
    "Pankow": "03",
    "Charlottenburg-Wilmersdorf": "04",
    "Spandau": "05",
    "Steglitz-Zehlendorf": "06",
    "Tempelhof-Schöneberg": "07",
    "Neukölln": "08",
    "Treptow-Köpenick": "09",
    "Marzahn-Hellersdorf": "10",
    "Lichtenberg": "11",
    "Reinickendorf": "12"
}
df["district_id"] = df["district"].map(district_id_map)

In [19]:
schema = [
    bigquery.SchemaField("station", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("line", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("latitude", "FLOAT64", mode="REQUIRED"),
    bigquery.SchemaField("longitude", "FLOAT64", mode="REQUIRED"),
    bigquery.SchemaField("postcode", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("neighborhood", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("district", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("district_id", "STRING", mode="REQUIRED"),
]


In [ ]:
# Define the table with schema
table_id = "cohesive-idiom-441010-a8.berlin_data.ubahn"
table = bigquery.Table(table_id, schema=schema)

# Create the table first with the schema (won't raise error if it exists)
client.create_table(table, exists_ok=True)

# Then upload the DataFrame to the table
job = client.load_table_from_dataframe(df, table_id)
job.result()
